<a href="https://colab.research.google.com/github/sungjindo/black_coal_price/blob/main/blackcoal_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 라이브러리 및 마운트

In [1]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import os

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
%matplotlib inline

# 통계 및 기계학습 관련 라이브러리
from scipy import stats    # 통계 관련 라이브러리
from sklearn.model_selection import train_test_split, RandomizedSearchCV    # 데이터 셋 분류 라이브러리
from sklearn.utils import resample    # 데이터 업샘플링&다운샘플링 라이브러리
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTEENN    # 복합샘플링 라이브러리
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler    # 데이터 스케일링 라이브러리
from statsmodels.stats.outliers_influence import variance_inflation_factor    # 변수 간 다중공선성 확인(VIF) 라이브러리
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, plot_roc_curve)    # 모델 정확도 검증 라이브러리
from sklearn.model_selection import cross_val_score    # validation  검증 라이브러리
from sklearn.linear_model import LogisticRegression    # 로지스틱 회귀 라이브러리
from sklearn.linear_model import Lasso    # 라쏘 라이브러리
from sklearn.linear_model import Ridge    # 릿지 라이브러리
import statsmodels.api as sm    # 통계분석 관련 라이브러리
from xgboost import XGBClassifier    # XGBoost 라이브러리
from sklearn.ensemble import RandomForestClassifier    # 랜덤포레스트 라이브러리
import warnings    # 경고문구 지우기



/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install xlrd

# 엑셀 read

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#DATA

In [ ]:
# df= df.rename(columns={'Unnamed: 0':'date'})

In [ ]:
# # 전체 컬럼명 확인

# #pd.set_option('display.max_seq_items', 10)
# col_names = df.columns.tolist()
# print(df.columns)

In [ ]:
# df1 = df.set_index("date")
# df1

In [ ]:
# df_up = df1.resample('W-SUN').ffill()
# df_up

# # 보통 ffill을 사용하거나 보간법을 시행한다.
# # 기간 참고 https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
# # 개념 참고 https://datascienceschool.net/01%20python/04.08%20%EC%8B%9C%EA%B3%84%EC%97%B4%20%EC%9E%90%EB%A3%8C%20%EB%8B%A4%EB%A3%A8%EA%B8%B0.html


In [ ]:
# df_up.to_csv('/content/gdrive/My Drive/Black_coal/data/blackcoalt_m_wtrue1.csv', index = True, encoding = 'utf-8-sig')

In [ ]:
# df2 = pd.read_excel("/content/gdrive/My Drive/Black_coal/data/black_coal.xlsx", sheet_name = [2] )

# df_1 / df_2

In [9]:
df_1 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/m_w.csv", encoding = 'utf-8-sig')
df_1

,date,KPPI,ACPI,KCPI,APPI,APPI2
0,2008-02-03,5.1,211.693,3.6,0.010,101.000000
1,2008-02-10,5.1,211.693,3.6,0.010,101.000000
2,2008-02-17,5.1,211.693,3.6,0.010,101.000000
3,2008-02-24,5.1,211.693,3.6,0.010,101.000000
4,2008-03-02,6.0,213.528,3.9,0.003,101.303000
...,...,...,...,...,...,...
666,2020-11-08,-0.5,260.229,0.6,0.003,124.674642
667,2020-11-15,-0.5,260.229,0.6,0.003,124.674642
668,2020-11-22,-0.5,260.229,0.6,0.003,124.674642
669,2020-11-29,-0.5,260.229,0.6,0.003,124.674642


In [10]:
df_2 = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/dataset/bdi.csv", encoding = 'utf-8-sig')
df_2

,Date,Price
0,2000-01-03,1318
1,2000-01-04,1320
2,2000-01-05,1329
3,2000-01-06,1351
4,2000-01-07,1368
...,...,...
8172,2022-05-19,3289
8173,2022-05-20,3344
8174,2022-05-21,3344
8175,2022-05-22,3344


In [11]:
df_2= df_2.rename(columns={'Price':'bdi'})

df_2['date']= pd.to_datetime(df_2['Date'])
df_2

,Date,bdi,date
0,2000-01-03,1318,2000-01-03
1,2000-01-04,1320,2000-01-04
2,2000-01-05,1329,2000-01-05
3,2000-01-06,1351,2000-01-06
4,2000-01-07,1368,2000-01-07
...,...,...,...
8172,2022-05-19,3289,2022-05-19
8173,2022-05-20,3344,2022-05-20
8174,2022-05-21,3344,2022-05-21
8175,2022-05-22,3344,2022-05-22


In [12]:
df_2 = df_2.drop(columns=['Date'], axis=1)
df_2

,bdi,date
0,1318,2000-01-03
1,1320,2000-01-04
2,1329,2000-01-05
3,1351,2000-01-06
4,1368,2000-01-07
...,...,...
8172,3289,2022-05-19
8173,3344,2022-05-20
8174,3344,2022-05-21
8175,3344,2022-05-22


In [13]:
df_2 = df_2.set_index("date")
df_2

,bdi
date,
2000-01-03,1318
2000-01-04,1320
2000-01-05,1329
2000-01-06,1351
2000-01-07,1368
...,...
2022-05-19,3289
2022-05-20,3344
2022-05-21,3344


# df_price


In [14]:
df_price = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/가격데이터.csv", encoding = 'utf-8-sig')
df_price

,Unnamed: 0,date,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut
0,0,2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67
1,1,2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96
2,2,2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45
3,3,2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50
4,4,2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68
...,...,...,...,...,...,...,...,...,...,...
698,698,2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42
699,699,2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74
700,700,2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80
701,701,2008-12-14,73.29,NaN,44.00,NaN,33.87,35.89,NaN,59.11


In [15]:
df_price = df_price.drop(columns=['Unnamed: 0'], axis=1)
df_price

,date,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut
0,2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67
1,2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96
2,2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45
3,2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50
4,2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68
...,...,...,...,...,...,...,...,...,...
698,2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42
699,2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74
700,2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80
701,2008-12-14,73.29,NaN,44.00,NaN,33.87,35.89,NaN,59.11


In [16]:
df_price = df_price.set_index('date')
df_price

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut
date,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68
...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80


## df_combine

In [17]:
df_left = pd.merge(df_price, df_1, left_on='date', right_on='date', how='left')
print(df_left)

           date  Coal_price    iron  Brent_fut  copper_fut  WTI_fut     WTI  \
0    2022-05-22      408.00  133.99     111.75      9453.5   110.98  112.44   
1    2022-05-15      417.25  133.66     112.55      9422.0   113.23  112.04   
2    2022-05-08      392.85  132.33     111.55      9159.0   110.49  110.30   
3    2022-05-01      379.50  136.91     112.39      9414.5   109.77  110.95   
4    2022-04-24      326.60  150.77     109.34      9769.5   104.69  104.78   
..          ...         ...     ...        ...         ...      ...     ...   
698  2009-01-04       75.07     NaN      44.42         NaN    40.83   40.92   
699  2008-12-28       74.01     NaN      46.91         NaN    46.34   46.16   
700  2008-12-21       73.09     NaN      38.37         NaN    37.71   37.59   
701  2008-12-14       73.29     NaN      44.00         NaN    33.87   35.89   
702  2008-12-07       74.04     NaN      46.41         NaN    46.28   46.37   

      Dubai  LNG_fut  KPPI     ACPI  KCPI   APPI   

In [18]:
df_left.isnull().sum()

date            0
Coal_price      0
iron           98
Brent_fut       0
copper_fut     84
WTI_fut         0
WTI             0
Dubai         281
LNG_fut         1
KPPI           76
ACPI           76
KCPI           76
APPI           76
APPI2          76
dtype: int64

In [19]:
df_left['date']= pd.to_datetime(df_left['date'])

df_left = df_left.set_index('date')
df_left

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,KCPI,APPI,APPI2
date,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,NaN,NaN,NaN
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,NaN,NaN,NaN
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,NaN,NaN,NaN
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,NaN,NaN,NaN
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,3.7,-0.019,98.676685
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,4.1,-0.022,100.587854
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,4.1,-0.022,100.587854


In [20]:
# df_LEFT_JOIN_2 = df_LEFT_JOIN.join(df_2, how = 'left') #디폴트는 이너 
# print(df_LEFT_JOIN_2)

# # Index 기준 합칠땐 join
# # merge는 단순 합치기

df_left2 = pd.merge(df_left, df_2, left_index = True ,right_index = True, how='left')
print(df_left2)

            Coal_price    iron  Brent_fut  copper_fut  WTI_fut     WTI  \
date                                                                     
2022-05-22      408.00  133.99     111.75      9453.5   110.98  112.44   
2022-05-15      417.25  133.66     112.55      9422.0   113.23  112.04   
2022-05-08      392.85  132.33     111.55      9159.0   110.49  110.30   
2022-05-01      379.50  136.91     112.39      9414.5   109.77  110.95   
2022-04-24      326.60  150.77     109.34      9769.5   104.69  104.78   
...                ...     ...        ...         ...      ...     ...   
2009-01-04       75.07     NaN      44.42         NaN    40.83   40.92   
2008-12-28       74.01     NaN      46.91         NaN    46.34   46.16   
2008-12-21       73.09     NaN      38.37         NaN    37.71   37.59   
2008-12-14       73.29     NaN      44.00         NaN    33.87   35.89   
2008-12-07       74.04     NaN      46.41         NaN    46.28   46.37   

             Dubai  LNG_fut  KPPI    

In [21]:
# df_2 = df_2.sort_values('date', ascending= False)
# df_2

In [22]:
df_2.index.dtype

dtype('<M8[ns]')

In [23]:
df_left2.index.dtype

dtype('<M8[ns]')

In [24]:
df_left2

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,KCPI,APPI,APPI2,bdi
date,,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,NaN,NaN,NaN,3344
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,NaN,NaN,NaN,3104
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,NaN,NaN,NaN,2718
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,NaN,NaN,NaN,2404
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,NaN,NaN,NaN,2307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,3.7,-0.019,98.676685,773
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,4.1,-0.022,100.587854,774
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,4.1,-0.022,100.587854,818


In [25]:
df_left2.isnull().sum()

Coal_price      0
iron           98
Brent_fut       0
copper_fut     84
WTI_fut         0
WTI             0
Dubai         281
LNG_fut         1
KPPI           76
ACPI           76
KCPI           76
APPI           76
APPI2          76
bdi             0
dtype: int64

# Electronic

In [26]:
df_ele = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/ele_pro.csv", encoding = 'utf-8-sig')
df_ele

,date,South Korea_Electricity from coal (TWh),South Korea_Electricity from gas (TWh),South KoreaElectricity from hydro (TWh),South Korea_Electricity from other renewables including bioenergy (TWh),South Korea_Electricity from solar (TWh),South Korea_Electricity from oil (TWh),South Korea_Electricity from wind (TWh),South Korea_Electricity from nuclear (TWh),Japan_Electricity from coal (TWh),...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
0,2000-01-01,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
1,2000-01-02,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2,2000-01-03,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
3,2000-01-04,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
4,2000-01-05,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,2021-12-27,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
8032,2021-12-28,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
8033,2021-12-29,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
8034,2021-12-30,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67


In [27]:
df_ele.isnull().sum()

date                                                                       0
South Korea_Electricity from coal (TWh)                                    0
South Korea_Electricity from gas (TWh)                                     0
South KoreaElectricity from hydro (TWh)                                    0
South Korea_Electricity from other renewables including bioenergy (TWh)    0
South Korea_Electricity from solar (TWh)                                   0
South Korea_Electricity from oil (TWh)                                     0
South Korea_Electricity from wind (TWh)                                    0
South Korea_Electricity from nuclear (TWh)                                 0
Japan_Electricity from coal (TWh)                                          0
Japan_Electricity from gas (TWh)                                           0
Japan_Electricity from hydro (TWh)                                         0
Japan_Electricity from other renewables including bioenergy (TWh)          0

In [28]:
df_ele['date']= pd.to_datetime(df_ele['date'])

df_ele = df_ele.set_index('date')
df_ele

,South Korea_Electricity from coal (TWh),South Korea_Electricity from gas (TWh),South KoreaElectricity from hydro (TWh),South Korea_Electricity from other renewables including bioenergy (TWh),South Korea_Electricity from solar (TWh),South Korea_Electricity from oil (TWh),South Korea_Electricity from wind (TWh),South Korea_Electricity from nuclear (TWh),Japan_Electricity from coal (TWh),Japan_Electricity from gas (TWh),...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-02,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-03,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-04,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
2000-01-05,107.49,28.18,3.97,0.08,0.00,29.26,0.02,103.52,204.84,222.25,...,0.59,16.74,390.23,55.96,76.99,1.686,0.01,29.16,1.58,15.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,310.39,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
2021-12-28,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,310.39,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67
2021-12-29,199.05,167.92,6.16,22.82,23.71,2.27,3.17,150.14,296.65,310.39,...,614.18,407.47,1250.46,58.11,171.27,0.000,68.34,4.86,68.03,42.67


In [29]:
df_left2

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,KCPI,APPI,APPI2,bdi
date,,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,NaN,NaN,NaN,3344
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,NaN,NaN,NaN,3104
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,NaN,NaN,NaN,2718
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,NaN,NaN,NaN,2404
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,NaN,NaN,NaN,2307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,3.7,-0.019,98.676685,773
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,4.1,-0.022,100.587854,774
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,4.1,-0.022,100.587854,818


In [30]:
# df_LEFT_JOIN_2 = df_LEFT_JOIN.join(df_2, how = 'left') #디폴트는 이너 
# print(df_LEFT_JOIN_2)

# # Index 기준 합칠땐 join
# # merge는 단순 합치기

df_left3 = pd.merge(df_left2, df_ele , left_index = True ,right_index = True, how='left')
print(df_left3)

            Coal_price    iron  Brent_fut  copper_fut  WTI_fut     WTI  \
date                                                                     
2022-05-22      408.00  133.99     111.75      9453.5   110.98  112.44   
2022-05-15      417.25  133.66     112.55      9422.0   113.23  112.04   
2022-05-08      392.85  132.33     111.55      9159.0   110.49  110.30   
2022-05-01      379.50  136.91     112.39      9414.5   109.77  110.95   
2022-04-24      326.60  150.77     109.34      9769.5   104.69  104.78   
...                ...     ...        ...         ...      ...     ...   
2009-01-04       75.07     NaN      44.42         NaN    40.83   40.92   
2008-12-28       74.01     NaN      46.91         NaN    46.34   46.16   
2008-12-21       73.09     NaN      38.37         NaN    37.71   37.59   
2008-12-14       73.29     NaN      44.00         NaN    33.87   35.89   
2008-12-07       74.04     NaN      46.41         NaN    46.28   46.37   

             Dubai  LNG_fut  KPPI    

In [31]:
df_left3

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2022-05-22,408.00,133.99,111.75,9453.5,110.98,112.44,107.17,136.67,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-15,417.25,133.66,112.55,9422.0,113.23,112.04,107.21,147.96,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-08,392.85,132.33,111.55,9159.0,110.49,110.30,107.23,148.45,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-01,379.50,136.91,112.39,9414.5,109.77,110.95,107.57,137.50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-04-24,326.60,150.77,109.34,9769.5,104.69,104.78,102.79,163.68,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-01-04,75.07,NaN,44.42,NaN,40.83,40.92,NaN,56.42,4.6,211.143,...,27.61,70.05,611.11,104.37,106.34,11.642,0.08,13.08,16.27,16.82
2008-12-28,74.01,NaN,46.91,NaN,46.34,46.16,NaN,54.74,5.5,210.228,...,13.10,68.39,573.12,85.88,115.17,9.302,0.06,15.27,14.38,15.23
2008-12-21,73.09,NaN,38.37,NaN,37.71,37.59,NaN,59.80,5.5,210.228,...,13.10,68.39,573.12,85.88,115.17,9.302,0.06,15.27,14.38,15.23


In [32]:
df_left3.isnull().sum()

Coal_price                                                                   0
iron                                                                        98
Brent_fut                                                                    0
copper_fut                                                                  84
WTI_fut                                                                      0
WTI                                                                          0
Dubai                                                                      281
LNG_fut                                                                      1
KPPI                                                                        76
ACPI                                                                        76
KCPI                                                                        76
APPI                                                                        76
APPI2                                               

In [33]:
df_f = df_left3.query('"2011-01-01"<= date <= "2019-12-31"')

In [34]:
df_f

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,Dubai,LNG_fut,KPPI,ACPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2019-12-29,67.50,93.41,68.60,6129.5,63.05,63.10,67.83,33.07,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-22,66.55,91.55,68.16,6214.0,61.72,61.74,64.82,33.13,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-15,66.35,91.74,66.14,6175.0,60.44,60.44,64.63,37.12,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-08,66.30,92.27,65.22,6130.0,60.07,59.78,64.12,35.17,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-01,66.40,88.56,64.39,5990.0,59.20,59.11,63.43,39.05,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,NaN,54.84,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,NaN,53.73,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,NaN,55.12,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [35]:
df_f.isnull().sum()

Coal_price                                                                   0
iron                                                                         0
Brent_fut                                                                    0
copper_fut                                                                   0
WTI_fut                                                                      0
WTI                                                                          0
Dubai                                                                      173
LNG_fut                                                                      0
KPPI                                                                         0
ACPI                                                                         0
KCPI                                                                         0
APPI                                                                         0
APPI2                                               

In [36]:
df_f = df_f.drop('Dubai', axis = 1)
df_f

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,LNG_fut,KPPI,ACPI,KCPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2019-12-29,67.50,93.41,68.60,6129.5,63.05,63.10,33.07,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-22,66.55,91.55,68.16,6214.0,61.72,61.74,33.13,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-15,66.35,91.74,66.14,6175.0,60.44,60.44,37.12,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-08,66.30,92.27,65.22,6130.0,60.07,59.78,35.17,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-01,66.40,88.56,64.39,5990.0,59.20,59.11,39.05,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,54.84,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,53.73,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,55.12,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [37]:
df_f.isnull().sum()

Coal_price                                                                 0
iron                                                                       0
Brent_fut                                                                  0
copper_fut                                                                 0
WTI_fut                                                                    0
WTI                                                                        0
LNG_fut                                                                    0
KPPI                                                                       0
ACPI                                                                       0
KCPI                                                                       0
APPI                                                                       0
APPI2                                                                      0
bdi                                                                        0

In [38]:
df_f.to_csv('/content/gdrive/My Drive/Black_coal/data/dataset/blackcoal_combine.csv', index = True, encoding = 'utf-8-sig')

#df_final

In [46]:
df_f = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/dataset/blackcoal_combine.csv", encoding = 'utf-8-sig')
df_f['date']= pd.to_datetime(df_f['date'])

df_f = df_f.set_index('date')
df_f

,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,LNG_fut,KPPI,ACPI,KCPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
date,,,,,,,,,,,,,,,,,,,,,
2019-12-29,67.50,93.41,68.60,6129.5,63.05,63.10,33.07,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-22,66.55,91.55,68.16,6214.0,61.72,61.74,33.13,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-15,66.35,91.74,66.14,6175.0,60.44,60.44,37.12,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-08,66.30,92.27,65.22,6130.0,60.07,59.78,35.17,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2019-12-01,66.40,88.56,64.39,5990.0,59.20,59.11,39.05,0.7,256.974,0.7,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,54.84,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,53.73,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,55.12,6.8,220.223,3.4,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [41]:
df_fin = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/finance data.csv", encoding = 'utf-8-sig')
df_fin

,Unnamed: 0,USD/AUD(호주),USD/CAD(캐나다),USD/COP(콜롬비아),US 달러지수,USD/IDR(인도네시아),USD/RUB(러시아),USD/ZAR(자메이카),상하이종합지수,인도지수,...,호주지수,캐나다지수,베트남GDP,인도GDP,한국GDP,일본GDP,중국GDP,미국GDP,EUGDP,세계GDP
0,2009-01-01,1.418400,1.22180,2284.100098,82.150002,11142.000000,29.142000,9.47760,1820.805054,9647.309570,...,3722.300049,8987.700195,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
1,2009-01-02,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2,2009-01-03,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
3,2009-01-04,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
4,2009-01-05,1.400000,1.18800,2219.899902,83.574997,10929.000000,28.943001,9.26630,1880.715942,10275.599609,...,3687.000000,9285.500000,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378,2020-12-27,1.314613,1.28482,3503.250000,90.250000,14148.643555,74.029999,14.53520,3396.562988,46973.539062,...,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
4379,2020-12-28,1.313730,1.28396,3513.250000,90.275002,13911.143555,73.954903,14.60990,3397.284912,47353.750000,...,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
4380,2020-12-29,1.317710,1.28400,3498.250000,89.917999,14126.120117,73.930496,14.62291,3379.035889,47613.078125,...,6700.299805,17543.400391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
4381,2020-12-30,1.314078,1.28221,3472.750000,89.649002,14069.909180,74.021400,14.71590,3414.452881,47746.218750,...,6682.399902,17545.800781,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13


In [47]:
df_fin = df_fin.rename(columns={'Unnamed: 0':'date'})
df_fin['date']= pd.to_datetime(df_fin['date'])


df_fin = df_fin.set_index('date')
df_fin

,USD/AUD(호주),USD/CAD(캐나다),USD/COP(콜롬비아),US 달러지수,USD/IDR(인도네시아),USD/RUB(러시아),USD/ZAR(자메이카),상하이종합지수,인도지수,브라질지수,...,호주지수,캐나다지수,베트남GDP,인도GDP,한국GDP,일본GDP,중국GDP,미국GDP,EUGDP,세계GDP
date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,1.418400,1.22180,2284.100098,82.150002,11142.000000,29.142000,9.47760,1820.805054,9647.309570,37550.0,...,3722.300049,8987.700195,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-02,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,40244.0,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-03,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,40244.0,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-04,1.409800,1.20940,2221.800049,82.805000,10991.000000,28.697001,9.28000,1820.805054,9958.219727,40244.0,...,3713.800049,9234.099609,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
2009-01-05,1.400000,1.18800,2219.899902,83.574997,10929.000000,28.943001,9.26630,1880.715942,10275.599609,41519.0,...,3687.000000,9285.500000,1.060147e+11,1.341887e+12,9.439419e+11,5.289493e+12,5.101703e+12,1.444893e+13,1.476378e+13,6.078101e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,1.314613,1.28482,3503.250000,90.250000,14148.643555,74.029999,14.53520,3396.562988,46973.539062,117857.0,...,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
2020-12-28,1.313730,1.28396,3513.250000,90.275002,13911.143555,73.954903,14.60990,3397.284912,47353.750000,119051.0,...,6664.799805,17623.900391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13
2020-12-29,1.317710,1.28400,3498.250000,89.917999,14126.120117,73.930496,14.62291,3379.035889,47613.078125,119475.0,...,6700.299805,17543.400391,2.711584e+11,2.660245e+12,1.637896e+12,5.057759e+12,1.472273e+13,2.095303e+13,1.529193e+13,8.474698e+13


In [48]:
df_final = pd.merge(df_f, df_fin , left_index = True ,right_index = True, how='left')
print(df_final)

            Coal_price    iron  Brent_fut  copper_fut  WTI_fut    WTI  \
date                                                                    
2019-12-29       67.50   93.41      68.60      6129.5    63.05  63.10   
2019-12-22       66.55   91.55      68.16      6214.0    61.72  61.74   
2019-12-15       66.35   91.74      66.14      6175.0    60.44  60.44   
2019-12-08       66.30   92.27      65.22      6130.0    60.07  59.78   
2019-12-01       66.40   88.56      64.39      5990.0    59.20  59.11   
...                ...     ...        ...         ...      ...    ...   
2011-01-30      127.00  183.50      99.83     10067.0    89.03  89.13   
2011-01-23      130.35  179.50      99.42      9548.5    89.34  89.49   
2011-01-16      131.90  178.00      97.60      9462.5    89.11  89.13   
2011-01-09      136.00  179.50      98.68      9662.0    91.54  91.66   
2011-01-02      136.35  173.50      93.33      9436.0    88.03  88.48   

            LNG_fut  KPPI     ACPI  KCPI  ...     

In [52]:
a = df_final.isnull().sum()
a

Coal_price    0
iron          0
Brent_fut     0
copper_fut    0
WTI_fut       0
             ..
일본GDP         0
중국GDP         0
미국GDP         0
EUGDP         0
세계GDP         0
Length: 75, dtype: int64

In [53]:
df_final.to_csv('/content/gdrive/My Drive/Black_coal/data/dataset/df_final.csv', index = True, encoding = 'utf-8-sig')

#train/test

In [57]:
df_f = pd.read_csv("/content/gdrive/My Drive/Black_coal/data/dataset/blackcoal_combine.csv", encoding = 'utf-8-sig')
df_f

,date,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,LNG_fut,KPPI,ACPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
0,2019-12-29,67.50,93.41,68.60,6129.5,63.05,63.10,33.07,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
1,2019-12-22,66.55,91.55,68.16,6214.0,61.72,61.74,33.13,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
2,2019-12-15,66.35,91.74,66.14,6175.0,60.44,60.44,37.12,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
3,2019-12-08,66.30,92.27,65.22,6130.0,60.07,59.78,35.17,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
4,2019-12-01,66.40,88.56,64.39,5990.0,59.20,59.11,39.05,0.7,256.974,...,405.3,348.7,1184.53,67.47,162.02,29.667,46.27,5.05,63.31,45.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,54.84,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
466,2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,53.73,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
467,2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,55.12,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
468,2011-01-09,136.00,179.50,98.68,9662.0,91.54,91.66,55.31,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [58]:
df_f['date'] = pd.to_datetime(df_f['date'])
df_f['date']

0     2019-12-29
1     2019-12-22
2     2019-12-15
3     2019-12-08
4     2019-12-01
         ...    
465   2011-01-30
466   2011-01-23
467   2011-01-16
468   2011-01-09
469   2011-01-02
Name: date, Length: 470, dtype: datetime64[ns]

In [59]:
print(df_f['date'].max())
print(df_f['date'].min())
print(df_f['date'].max() - df_f['date'].min())

2019-12-29 00:00:00
2011-01-02 00:00:00
3283 days 00:00:00


In [60]:
train = df_f[(df_f['date'] < '2019-01-01')]
lag_size = (train['date'].max() - train['date'].min())
print('Min date from train set : %s' % train['date'].min().date())
print('Max date from train set : %s' % train['date'].max().date())
print('Forecast lag size', lag_size)

Min date from train set : 2011-01-02
Max date from train set : 2018-12-30
Forecast lag size 2919 days 00:00:00


In [61]:
train

,date,Coal_price,iron,Brent_fut,copper_fut,WTI_fut,WTI,LNG_fut,KPPI,ACPI,...,China_Electricity from wind (TWh),China_Electricity from nuclear (TWh),India_Electricity from coal (TWh),India_Electricity from gas (TWh),India_Electricity from hydro (TWh),India_Electricity from other renewables including bioenergy (TWh),India_Electricity from solar (TWh),India_Electricity from oil (TWh),India_Electricity from wind (TWh),India_Electricity from nuclear (TWh)
52,2018-12-30,99.50,73.09,57.06,5918.0,47.96,48.18,63.55,0.9,251.233,...,365.8,295.0,1198.86,71.26,139.75,27.224,36.33,6.20,60.31,39.05
53,2018-12-23,101.90,69.20,52.20,5997.0,45.33,44.92,62.63,0.9,251.233,...,365.8,295.0,1198.86,71.26,139.75,27.224,36.33,6.20,60.31,39.05
54,2018-12-16,101.70,68.99,53.82,5991.0,45.59,45.08,69.05,0.9,251.233,...,365.8,295.0,1198.86,71.26,139.75,27.224,36.33,6.20,60.31,39.05
55,2018-12-09,102.30,67.89,60.28,6131.5,51.20,51.17,67.95,0.9,251.233,...,365.8,295.0,1198.86,71.26,139.75,27.224,36.33,6.20,60.31,39.05
56,2018-12-02,102.55,66.76,61.67,6145.0,52.61,52.05,63.30,0.9,251.233,...,365.8,295.0,1198.86,71.26,139.75,27.224,36.33,6.20,60.31,39.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,2011-01-30,127.00,183.50,99.83,10067.0,89.03,89.13,54.84,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
466,2011-01-23,130.35,179.50,99.42,9548.5,89.34,89.49,53.73,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
467,2011-01-16,131.90,178.00,97.60,9462.5,89.11,89.13,55.12,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22
468,2011-01-09,136.00,179.50,98.68,9662.0,91.54,91.66,55.31,6.8,220.223,...,74.1,87.2,701.25,117.56,131.70,17.137,0.83,9.35,23.95,32.22


In [64]:
test = df_f[(df_f['date'] >= '2019-01-01')]
lag_size = (test['date'].max() - test['date'].min())
print('Min date from test set : %s' % test['date'].min().date())
print('Max date from test set : %s' % test['date'].max().date())
print('Forecast lag size', lag_size)

Min date from test set : 2019-01-06
Max date from test set : 2019-12-29
Forecast lag size 357 days 00:00:00


In [66]:
test_data = test.drop('Coal_price', axis = 1)
test_price = test['Coal_price']
print(test_data.head())
print(test_price.head())

        date   iron  Brent_fut  copper_fut  WTI_fut    WTI  LNG_fut  KPPI  \
0 2019-12-29  93.41      68.60      6129.5    63.05  63.10    33.07   0.7   
1 2019-12-22  91.55      68.16      6214.0    61.72  61.74    33.13   0.7   
2 2019-12-15  91.74      66.14      6175.0    60.44  60.44    37.12   0.7   
3 2019-12-08  92.27      65.22      6130.0    60.07  59.78    35.17   0.7   
4 2019-12-01  88.56      64.39      5990.0    59.20  59.11    39.05   0.7   

      ACPI  KCPI  ...  China_Electricity from wind (TWh)  \
0  256.974   0.7  ...                              405.3   
1  256.974   0.7  ...                              405.3   
2  256.974   0.7  ...                              405.3   
3  256.974   0.7  ...                              405.3   
4  256.974   0.7  ...                              405.3   

   China_Electricity from nuclear (TWh)  India_Electricity from coal (TWh)  \
0                                 348.7                            1184.53   
1                   

In [ ]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
    
train_gp = train.sort_values('date').groupby(['item', 'store', 'date'], as_index=False)
train_gp = train_gp.agg({'sales':['mean']})
train_gp.columns = ['item', 'store', 'date', 'sales']

window = 29
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)

last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)